In [1]:

# import zipfile

# # Ruta del archivo ZIP
# zip_file_path = '/content/tesis-main.zip'

# # Ruta donde quieres extraer los archivos
# extract_to_path = '/content/'

# # Descomprimir el archivo
# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_to_path)

# print("Archivos extraídos con éxito.")

In [2]:
import sys
sys.path.append('/content/tesis-main/')

from dotenv import load_dotenv

load_dotenv()

True

In [3]:

# import drive as drive_downloader
# drive_downloader.download_files('14_UUqgoWDPPNwXvdFjDoJomHTu5l85l6')

* Carga el video utilizando cv2.VideoCapture.
* Extrae los frames del video uno por uno.
* Preprocesa cada frame para que sean compatibles con el modelo de PyTorch (tamaño, normalización, etc.).
* Usa un modelo preentrenado de PyTorch (como ResNet) para extraer características de cada frame.
* Aplica un algoritmo de clustering (como K-Means) a las características extraídas.
* Asigna cada frame al cluster correspondiente.
* Visualiza los resultados mostrando los frames y su cluster correspondiente.

In [4]:
import torch
from sklearn.cluster import AgglomerativeClustering
import warnings
from models.resnet18 import get_model as get_resnet18
from preprocessing.transforms import BASELINE
from clustering.metrics import eval_massive_cluster
from clustering.model import clustering_function
from experiment_framework import experiment
from validation import VALIDATION_DATASET
from utils import load_dataset
warnings.filterwarnings('ignore')
import wandb

torch.manual_seed(47)
import random
random.seed(47)
import numpy as np
np.random.seed(47)

if torch.cuda.is_available():
 dev = "cuda:0"
else:
 dev = "cpu"


device = torch.device(dev)
epochs = 10
steps=1
clustering_eval_function = lambda cluster_labels, tags: eval_massive_cluster(cluster_labels, tags, steps, epochs)

dataset = load_dataset(VALIDATION_DATASET.keys())

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mandirolamarcelo (hortensias). Use `wandb login --relogin` to force relogin


True

In [5]:
from properties import MODELS_ENUM, PREPROCESSORS, ALGORITHM, GROUPER_FUNCTIONS

DEFAULT_PARAMS = {
      "dataset": dataset,
      "evaluation_function": clustering_eval_function,
      "show": True,
      "device": device
   }


EXPERIMENT_DEFINITIONS= [
    {
        'name': 'ResnetAglomerative',
        'model': "RESNET18",
        'algorithm': "AGGLOMERATIVE",
        'params': {"n_clusters": None, "distance_threshold": 50},
        'grouper_function': "CLUSTERING_FUNCTION",
        'preprocessing': "BASELINE",
    },
    {
        'name': 'SimilarityClustering',
        'model': "RESNET18",
        'algorithm': "SIMILARITY",
        'params': {"threshold": 0.999},
        'grouper_function': "CLUSTERING_FUNCTION",
        'preprocessing': "BASELINE",
    }
]

for exp_definition in EXPERIMENT_DEFINITIONS:
  params = DEFAULT_PARAMS.copy()
  params["model"] = MODELS_ENUM[exp_definition["model"]](device)
  params["preprocessing"] = PREPROCESSORS[exp_definition["preprocessing"]]
  algorithm = ALGORITHM[exp_definition["algorithm"]](**exp_definition["params"])
  grouper_function = GROUPER_FUNCTIONS[exp_definition["grouper_function"]](algorithm)
  params["grouper_function"] = grouper_function
  params["name"] = exp_definition["name"]

  run = wandb.init(project="ort-tesis",group=exp_definition['name'],config=exp_definition, entity="hortensias")

  result = experiment(**params)
  print(result._asdict())
  run.log({
    "Precision": result.precision.mean,
    "Accuracy": result.accuracy.mean,
    "F1": result.f1.mean,
    "Recall": result.recall.mean,
    "Precision_STD": result.precision.std,
    "Accuracy_STD": result.accuracy.std,
    "F1_STD": result.f1.std,
    "Recall_STD": result.recall.std,
  })
  run.finish()


Experiment name: ResnetAglomerative Airbnb.mp4
Precision - Mean: 0.61, std: 0.0426408229
Recall - Mean: 0.50, std: 0.0542994165
F1-Score - Mean: 0.51, std: 0.0533845729
Accuracy - Mean: 0.50, std: 0.0542994165

Experiment name: ResnetAglomerative AVG
Precision - Mean: 0.61, std: 0.0000000000
Recall - Mean: 0.50, std: 0.0000000000
F1-Score - Mean: 0.51, std: 0.0000000000
Accuracy - Mean: 0.50, std: 0.0000000000
{'precision': MetricsResults(mean=0.6109762163769819, std=0.0), 'recall': MetricsResults(mean=0.5017724413950829, std=0.0), 'f1': MetricsResults(mean=0.5101993366100538, std=0.0), 'accuracy': MetricsResults(mean=0.5017724413950829, std=0.0)}


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Accuracy,▁
Accuracy_STD,▁
F1,▁
F1_STD,▁
Precision,▁
Precision_STD,▁
Recall,▁
Recall_STD,▁
Accuracy,0.50177
Accuracy_STD,0.0
F1,0.5102



Experiment name: SimilarityClustering Airbnb.mp4
Precision - Mean: 0.94, std: 0.0000000000
Recall - Mean: 0.49, std: 0.0000000000
F1-Score - Mean: 0.61, std: 0.0000000000
Accuracy - Mean: 0.49, std: 0.0000000000

Experiment name: SimilarityClustering AVG
Precision - Mean: 0.94, std: 0.0000000000
Recall - Mean: 0.49, std: 0.0000000000
F1-Score - Mean: 0.61, std: 0.0000000000
Accuracy - Mean: 0.49, std: 0.0000000000
{'precision': MetricsResults(mean=0.9399112694007534, std=0.0), 'recall': MetricsResults(mean=0.4905660377358491, std=0.0), 'f1': MetricsResults(mean=0.6145718925642055, std=0.0), 'accuracy': MetricsResults(mean=0.4905660377358491, std=0.0)}


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Accuracy,▁
Accuracy_STD,▁
F1,▁
F1_STD,▁
Precision,▁
Precision_STD,▁
Recall,▁
Recall_STD,▁
Accuracy,0.49057
Accuracy_STD,0.0
F1,0.61457
